## Milestone 4

Goals: 
- Train model for sentiment analysis

In [ ]:
!wget -q https://github.com/idmmachinelearning-ms/Project/blob/main/projectutils.py

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install datasets transformers huggingface_hub

!apt-get install git-lfs

import datasets
from datasets import load_dataset, Dataset, DatasetDict

from transformers import AutoTokenizer, pipeline




[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# one cell to create df that I can bring into final project

updatedpoems = open("./poetrydatasetml.txt", encoding='utf8').read()
all_lines = updatedpoems.split("\n\n\n")

titles = []
lines = []
words = []
length = []

for t in all_lines:
    a = t.split("\n\n")
    titles.append(a[0])
    lines.append(a[1:])

titles[0] = "Introduction"

for w in lines:
    a = " ".join(w)
    b = a.replace("\n", " ")
    c = b.split(" ")
    words.append(c)

for w in words:
    length.append(len(w))
    

poems_df = pd.DataFrame(titles)
poems_df["Lines"] = lines
poems_df["Words"] = words
poems_df["Length"] = length

poems_df.columns = ["Title", "Lines", "Words", " Word Length"]


In [9]:
poems_df.head()

,Title,Lines,Words,Word Length
0,Introduction,"[Introduction, Cycles can be stupid. You know ...","[Introduction, Cycles, can, be, stupid., You, ...",550
1,Whenever it’s convenient!,[Screaming\nI want you to see me\nPlease could...,"[Screaming, I, want, you, to, see, me, Please,...",27
2,Larger than Life,"[Biding my time,\nI am in no rush.\nPaintbrush...","[Biding, my, time,, I, am, in, no, rush., Pain...",180
3,Google Keep,[poems that sit in notes on my phone\nlittered...,"[poems, that, sit, in, notes, on, my, phone, l...",72
4,Google Keep 2,[poems that sit in notes on my phone\nscribble...,"[poems, that, sit, in, notes, on, my, phone, s...",92


In [10]:
# will try this https://huggingface.co/blog/sentiment-analysis-python
# https://discuss.huggingface.co/t/from-pandas-dataframe-to-huggingface-dataset/9322/4


huggingfacepoem = Dataset.from_pandas(poems_df)

#ds = DatasetDict()

#ds['train'] = huggingfacepoem

#print(ds)
print(huggingfacepoem)

Dataset({
    features: ['Title', 'Lines', 'Words', ' Word Length'],
    num_rows: 164
})


In [7]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
small_train_dataset = huggingfacepoem.shuffle(seed=42).select([i for i in list(range(100))])

In [9]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenized_train = huggingfacepoem.map(preprocess_function, batched=True)

In [ ]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)

In [48]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [ ]:
poems_encoded = huggingfacepoem.map(tokenize, batched=True, batch_size=None)

Both times, I am getting an issue whewn it comes to encoding the data ('text'). Perhaps there is something wrong with the way I transferred over my data to hugging face dataset that is causing an issue? Perhaps there is too much text with multiple columns in the dataframe? Perhaps there is too much text in each array belonging to each poem?



In [3]:
smaller_dataset = poems_df["Words"]
smaller_dataset

0      [Introduction, Cycles, can, be, stupid., You, ...
1      [Screaming, I, want, you, to, see, me, Please,...
2      [Biding, my, time,, I, am, in, no, rush., Pain...
3      [poems, that, sit, in, notes, on, my, phone, l...
4      [poems, that, sit, in, notes, on, my, phone, s...
                             ...                        
159    [this, pain, is, a, reminder, of, your, capaci...
160    [i, , have, this, gnawing, fear, of, the, end,...
161    [i, have, always, sought, a, mirror, a, portal...
162    [admiration, hand, in, hand, with, hesitation,...
163    [You, think, I’d, be, angry, but, I, am, not.,...
Name: Words, Length: 164, dtype: object

In [4]:
for entry in smaller_dataset:
    sentiment_pipeline = pipeline("sentiment-analysis")
    sentiment_pipeline(entry)



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
No model was supplied, defaulted to distilb

This took 6 min to complete (since it had to go through each one) but it seems it worked! Let's try refining it. I didn't specify the model but the distilbert-base-uncased-emotion is the model I want for detecting emotion. 

In [14]:
for entry in smaller_dataset[:3]:
    sentiment_pipeline = pipeline("sentiment-analysis")
    print(sentiment_pipeline(entry))
    

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9988112449645996}, {'label': 'POSITIVE', 'score': 0.8069758415222168}, {'label': 'POSITIVE', 'score': 0.9984523057937622}, {'label': 'POSITIVE', 'score': 0.9619516730308533}, {'label': 'NEGATIVE', 'score': 0.9997432827949524}, {'label': 'POSITIVE', 'score': 0.9991247057914734}, {'label': 'POSITIVE', 'score': 0.9947928786277771}, {'label': 'POSITIVE', 'score': 0.9942895174026489}, {'label': 'POSITIVE', 'score': 0.9797317385673523}, {'label': 'NEGATIVE', 'score': 0.999183714389801}, {'label': 'POSITIVE', 'score': 0.9983267188072205}, {'label': 'POSITIVE', 'score': 0.9953711628913879}, {'label': 'POSITIVE', 'score': 0.993144154548645}, {'label': 'POSITIVE', 'score': 0.9274746775627136}, {'label': 'POSITIVE', 'score': 0.9934518337249756}, {'label': 'POSITIVE', 'score': 0.9977535605430603}, {'label': 'POSITIVE', 'score': 0.9564287662506104}, {'label': 'POSITIVE', 'score': 0.9652853012084961}, {'label': 'POSITIVE', 'score': 0.9996235370635986}, {'label': 'PO

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9994392991065979}, {'label': 'POSITIVE', 'score': 0.9797317385673523}, {'label': 'POSITIVE', 'score': 0.9870406985282898}, {'label': 'POSITIVE', 'score': 0.9991247057914734}, {'label': 'POSITIVE', 'score': 0.9652853012084961}, {'label': 'POSITIVE', 'score': 0.998429000377655}, {'label': 'POSITIVE', 'score': 0.9947214126586914}, {'label': 'POSITIVE', 'score': 0.9995015859603882}, {'label': 'POSITIVE', 'score': 0.8508517742156982}, {'label': 'POSITIVE', 'score': 0.9991247057914734}, {'label': 'POSITIVE', 'score': 0.8226318955421448}, {'label': 'NEGATIVE', 'score': 0.998464822769165}, {'label': 'POSITIVE', 'score': 0.9982263445854187}, {'label': 'NEGATIVE', 'score': 0.992764413356781}, {'label': 'POSITIVE', 'score': 0.9797317385673523}, {'label': 'POSITIVE', 'score': 0.9995844960212708}, {'label': 'POSITIVE', 'score': 0.9926723837852478}, {'label': 'POSITIVE', 'score': 0.9129678606987}, {'label': 'POSITIVE', 'score': 0.9972309470176697}, {'label': 'POSITI

In [15]:
smaller_dataset[:3]

0    [Introduction, Cycles, can, be, stupid., You, ...
1    [Screaming, I, want, you, to, see, me, Please,...
2    [Biding, my, time,, I, am, in, no, rush., Pain...
Name: Words, dtype: object

Based on the analysis, we can take a look at some sentiment analysis done by the pre-trained model.

- Introduction {'label': 'POSITIVE', 'score': 0.9988112449645996}
- Cycles {'label': 'POSITIVE', 'score': 0.8069758415222168}
- can {'label': 'POSITIVE', 'score': 0.9984523057937622}
- be {'label': 'POSITIVE', 'score': 0.9619516730308533}
- stupid {'label': 'NEGATIVE', 'score': 0.9997432827949524}


- Screaming {'label': 'NEGATIVE', 'score': 0.9994392991065979}
- I {'label': 'POSITIVE', 'score': 0.9797317385673523}
- want {'label': 'POSITIVE', 'score': 0.9870406985282898}
- you {'label': 'POSITIVE', 'score': 0.9991247057914734}
- to {'label': 'POSITIVE', 'score': 0.9652853012084961}
- see {'label': 'POSITIVE', 'score': 0.998429000377655}
- me {'label': 'POSITIVE', 'score': 0.9947214126586914}


- Biding {'label': 'POSITIVE', 'score': 0.8047969341278076}
- my {'label': 'POSITIVE', 'score': 0.9934518337249756}
- time {'label': 'POSITIVE', 'score': 0.9797317385673523}
- I {'label': 'POSITIVE', 'score': 0.9546941518783569}
- am {'label': 'POSITIVE', 'score': 0.9789573550224304}
- in {'label': 'NEGATIVE', 'score': 0.9964459538459778},
- no {'label': 'NEGATIVE', 'score': 0.9977946281433105}


I think the model takes more neutral words and deems them positive. The scoring could be a fun thing to add into a dataframe and then do analysis. But then the data frame would have to change to have each word as an entry rather than a poem. Perhaps the poem could be likened to a label, like we have done in other exercises. This could give more opportunity to look at covariances. Maybe the new goal is to cluster poems together based on similar sentiment overall scores? 


#### IN PROGRESS FOR MILESTONE 5. 
Let's look at RNNs based on the Week 14 notebook.

In [ ]:
from random import choice
from torch import nn, Tensor
from torch.utils.data import DataLoader
from projectutils import TextSequenceDataset


In [13]:
class NGramDataset(TextSequenceDataset):
  def __init__(self, text, max_words=200_000, window=2):
    super().__init__(text, max_words, window, symmetric_context=False)
    self.words_t = self.encode(self.words)

  def __len__(self):
    return len(self.words) - self.window

  def __getitem__(self, idx):
    target = self.words_t[idx + self.window]
    context = self.words_t[idx : idx + self.window]
    return context, target